In [ ]:
import librosa
import numpy as np
from numpy.fft import rfft
from numpy import pi
from matplotlib import pyplot as plt
from IPython.display import Audio
import cmath
import scipy

TWO_PI = np.pi * 2

In [ ]:
PAGE_LEN = 1024
SR = 22050
DTYPE = np.float32

In [ ]:
HANN = scipy.signal.get_window('hann', PAGE_LEN, True)
IMAGINARY_LADDER = np.linspace(0, TWO_PI * 1j, PAGE_LEN)
SPECTRUM_SIZE = PAGE_LEN // 2 + 1
NYQUIST = SR // 2

In [ ]:
def sino(freq, length):
    return np.sin(np.arange(length) * freq * TWO_PI / SR)

def playHard(data):
    return Audio(data, rate = SR)
def play(data, soft = .1):
    t = np.concatenate([data, [1]])
    length = round(soft * SR)
    t[:length ] = np.multiply(t[:length ], np.linspace(0, 1, length))
    t[-length:] = np.multiply(t[-length:], np.linspace(1, 0, length))
    return playHard(t)

def sft(signal, freq_bin):
    # Slow Fourier Transform
    return np.abs(np.sum(signal * np.exp(IMAGINARY_LADDER * freq_bin))) / PAGE_LEN

def widePlot(h = 3, w = 12):
    plt.gcf().set_size_inches(w, h)
    
def spectro(signal, do_wide = True, trim = 130):
    energy = np.abs(rfft(signal * HANN))
    plt.plot(energy[:trim])
    if do_wide:
        widePlot()

def concatSynth(synth, harmonics, n):
    buffer = []
    for i in range(n):
        synth.eat(harmonics)
        buffer.append(synth.mix())
    return np.concatenate(buffer)

def pitch2freq(pitch):
    return np.exp((pitch + 36.37631656229591) * 0.0577622650466621)

def freq2pitch(f):
    return np.log(f) * 17.312340490667562 - 36.37631656229591

def pagesOf(signal):
    for i in range(0, signal.size - PAGE_LEN + 1, PAGE_LEN):
        yield signal[i : i + PAGE_LEN]


In [ ]:
from time import sleep
import mido

In [ ]:
mido.get_output_names()

In [ ]:
port = mido.open_output(
    # 'CASIO USB-MIDI 2', 
    'ARIUS:ARIUS MIDI 1', 
)
port

In [ ]:
port.send(mido.Message('note_on', note=60))

In [ ]:
# port.send(mido.Message('note_off', note=60))
port.panic()

In [ ]:
hb = [
    0, 0, 2, None, 0, None, 5, None, 4, None, None, None, 
    0, 0, 2, None, 0, None, 7, None, 5, None, None, None, 
    0, 0, 12, None, 9, None, 5, None, 4, None, 2, None, 
    10, 10, 9, None, 5, None, 7, None, 5, None, None, None,     
]

In [ ]:
def identity(x): return x

def playSong(song, interval = .2, middle = 55, func = identity):
    for note in song:
        if note is not None:
            func(note + middle)
        sleep(interval)
    port.panic()

In [ ]:
port.panic()

In [ ]:
playSong(hb)

## math

In [ ]:
from scipy.optimize import minimize
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
def getProduced(powers, n_partials):
    produced = np.zeros((n_partials, ))
    for i, power in enumerate(powers):
        n = i + 2
        for j in range(n, n_partials, n):
            produced[j] += power
    return produced
def loss(powers, n_partials):
    produced = getProduced(powers, n_partials)[2:]
    if 0 in produced:
        return np.inf
    return np.sum(np.square(np.log(produced)))

In [ ]:
def optim(n_partials, n_seeds = 100):
    results = []
    for _ in range(n_seeds):
        guess = np.random.rand(n_partials - 2)
        result = minimize(
            loss, guess, 
            args = (n_partials, ), 
            bounds = [(0, None)] * len(guess), 
        )
        results.append((result.fun, result.x))
    results = sorted(results, key=lambda x:x[0])[:round(.1 * n_seeds)]
    for y, x in results:
        plt.plot(x)
    widePlot()
    plt.show()
    return results

In [ ]:
results = optim(50)

In [ ]:
losses = [x[0] for x in results]
losses
# plt.plot(losses)
# plt.ylim(bottom=0)
# plt.show()

In [ ]:
from prime import getPrimesFrom3

In [ ]:
getPrimesFrom3(15)

In [ ]:
def present(results, n_partials):
    y, x = results[0]
    print('y', y)
    print('x')
    print(x)
    for prime in [2, *getPrimesFrom3(n_partials)]:
        plt.axvline(prime, c='r')
    plt.plot([0, 0, *x])
    plt.plot(getProduced(x, n_partials))
    widePlot()

In [ ]:
n_partials=50
results_50 = optim(n_partials, 10)
present(results_50, n_partials)
print('50 loss', loss(results_50[0][1], n_partials))

In [ ]:
n_partials=100
results_100 = optim(n_partials, 10)
present(results_100, n_partials)
print('50 loss', loss(results_100[0][1], 50))

In [ ]:
golden = results_50[0][1]
golden.size

In [ ]:
import random

import pretty_midi

In [ ]:
def computeOneGoldenNote(
    pitch, velocity = 64, n_partials = 50, 
    max_pitch = 127, 
):
    results = []
#     for i in reversed(range(2, N)):
    for partial_i in range(2, n_partials):
        delta_pitch = np.log(partial_i) * 17.312340490667562  # freq 2 pitch
        round_d_pitch = round(delta_pitch)
        played_pitch = pitch + round_d_pitch
        if played_pitch > max_pitch:
            continue
        residual = delta_pitch - round_d_pitch
        freq_err_adj = np.exp(- abs(residual) * 7)
        if pitch + round_d_pitch > 95:
            freq_err_adj *= .2
#         sub_energy = golden[i - 2] * velocity ** 2
#         adj_velocity = round((sub_energy * freq_err_adj) ** .5)
        sub_energy = golden[partial_i - 2] * velocity
        adj_velocity = round((sub_energy * freq_err_adj))
        if adj_velocity != 0:
            results.append((played_pitch, adj_velocity))
    return results

In [ ]:
def oneGoldenNote(pitch, velocity = 64, N = 50):
    port.panic()
    results = computeOneGoldenNote(pitch, velocity, N)
    random.shuffle(results)
    [port.send(mido.Message(
        'note_on', note=p, velocity=v, 
    )) for p, v in results]


In [ ]:
port.panic()
oneGoldenNote(48)

In [ ]:
port.panic()

In [ ]:
playSong(hb, interval = .15, middle = 48, func = oneGoldenNote)

Midi file

In [ ]:
def writeMidiBenchmark(
    pitch = 52, velocity = 127, interval = 0.4, padding = 0.1,
):
    mid = pretty_midi.PrettyMIDI()
    piano = pretty_midi.Instrument(0)
    mid.instruments.append(piano)
    cursor = 0.0
    piano.notes.append(pretty_midi.Note(
        pitch = pitch + 12,
        velocity = velocity,
        start = cursor, 
        end = cursor + interval, 
    ))
    cursor += interval + padding

    results = computeOneGoldenNote(pitch, velocity)
    print(*results, sep='\n')
    for p, v in results:
        piano.notes.append(pretty_midi.Note(
            pitch = p,
            velocity = v,
            start = cursor, 
            end = cursor + interval, 
        ))
    cursor += interval + padding
    
    mid.write('./temp/baseline.mid')
writeMidiBenchmark()